## The method to load circuits in `QASM` format.
?

# Explanation to the Optimization Techniques
## Pauli String Grouping Technique
### Abstract
One of the main methods proposed to suppress an increase in the number of measurements is the partitioning method, where Pauli strings are partitioned so that their expectation values can be measured simultaneously. 

In this method, a group of Pauli strings is divided into subgroups, and all the components of each subgroup are measured simultaneously using only one circuit (hereafter, such subgroups are called “partitions”). 

### Implementation
The problem of Pauli string grouping can be broken down into two parts:
1. The partition of a given group of Pauli strings.
2. The method to get the equivalent measurement results of pauli strings in a partition from the result of the so-called "simultaneous measurement" 

We solve the first part (the partition) of the problem by (?xxx L) calling the function/method in the `Estimator`, and we tackle the second part by using the function xxx in xxx.py

this is the workflow of xxx:
1. xxx



## Error Mitigation Technique


### Readout Error Mitigation

### Zero-Noise Extrapolation


## Transpile Optimization

# Alignment with the qiskit estimator
